In [1]:
from shapely.geometry import Point, Polygon
import fiona 
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
pd.set_option('display.max_colwidth', -1)

%run ../python_files/data_cleaning

In [2]:
#import data
wards = gpd.read_file("../data/google_maps/gis/Wards.kml", driver='KML')
census_tracts_gis = gpd.read_file("../data/open_data/gis/Census_Tracts_in_2010.kml", driver='KML')
puds_gis = gpd.read_file("../data/open_data/gis/Planned_Unit_Development_PUDs.kml", driver='KML')
tifs_gis = gpd.read_file("../data/open_data/gis/Tax_Increment_Financing_TIF_areas.kml", driver='KML')
evict_data = pd.read_csv("../data/eviction_lab/csv/tracts.csv")
census_tracts = pd.read_csv('../data/open_data/csv/Census_Tracts_in_2010.csv')
puds = pd.read_csv('../data/open_data/csv/Planned_Unit_Development_PUDs.csv') 
tifs = pd.read_csv("../data/open_data/csv/Tax_Increment_Financing_TIF_areas.csv")
puds_xwalk = pd.read_csv('../data/grassroots_dc/csv/zoning_crosswalk.csv')
puds_gr = pd.read_csv('../data/grassroots_dc/csv/PUD Data merged with public database - PUDs.csv')

In [3]:
#make tracts
evict16 = clean_eviction_data(evict_data)
tracts = merge_tracts(census_tracts_gis, census_tracts)
evict = merge_eviction_data(tracts, evict16)
evict = add_gis_ft1(wards, evict, 'ward')

In [4]:
#make puds
puds = merge_gis(puds_gis, puds)
puds_gr = clean_grassroots_pud_data(puds_gr)
puds = add_pud_ft(puds, puds_xwalk, puds_gr)
puds = add_gis_ft2(tracts, puds, 'GEOID')

In [6]:
#make tifs
tifs = merge_gis(tifs_gis, tifs)
tifs = add_gis_ft2(tracts, tifs, 'GEOID')

In [7]:
#make master files
master_puds = add_dev(puds, evict)
master_tifs = add_dev(tifs, evict)